<img src="img/dsci513_header2.png" width="600">

# Lab 2: Grouping, joins, and table manipulation

**Arman Seyed-Ahmadi, November 2021**

## Instructions
---
rubric={mechanics:2}

- Follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/)
- Make sure to upload a PDF version of your lab notebook to Gradescope, in addition to the `.ipynb` file. Use the `Webpdf` option of Jupyter Lab if `PDF` doesn't work.
- Add a link to your GitHub repository here:

## Imports and configurations
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import csv

%matplotlib inline
%load_ext sql
%config SqlMagic.displaylimit = 30
%config SqlMagic.autolimit = 30

## Connecting to the database
---

In [2]:
import json
import urllib.parse

with open('data/credentials.json') as f:
    login = json.load(f)
    
username = login['user']
password = urllib.parse.quote(login['password'])
host = login['host']
port = login['port']

## Exercise 1: Aggregations and grouping
---

For this exercise, use the `pd.read_sql_query` function from Pandas to execute your queries. But we need to use the `psycopg2` package to establish a connection to the `world_dsci513` database first:

In [3]:
conn = psycopg2.connect(database='world_dsci513', **login)

Because we read query results into Pandas dataframes, you'll see an **index column** appearing in your results. That is fine.

### 1.1

rubric={accuracy:2}

Write a query to answer the following question:

How much higher is the population of the most populated country in the world, with respect to the average population of all countries in the world, expressed in percent?

- You can find this value using the the formula $(\text{pop} - \text{pop}_\text{avg}) / \text{pop}_\text{avg}) \times 100$.
- Your query should print the value with only one digit after the decimal point, followed by the percent sign `%`, e.g. `2500.0%`.

In [4]:
query = """
SELECT
    ROUND(
        ((MAX(population) - AVG(population)) / AVG(population)) * 100, 1)
    || '%'
FROM
    country
;
"""

pd.read_sql_query(query, con=conn)

,?column?
0,4923.0%


### 1.2

rubric={accuracy:2}

Write a query to answer the following question:

What is the maximum, average, and minimum population density (population per surface area [person / km$^2$]) of countries located in Europe?

- The values in the `surfacearea` column already have the required unit (km$^2$); no unit conversion is required.
- Your column headers should read _Max pop_density_, _Average pop_density_, and _Min pop_density_.
- Round all values to 2 decimal digits.

>**Note:** Remember that you have to convert approximate types (e.g. double precision or real) to `NUMERIC` to be able to use the `ROUND()` function.

In [5]:
query = """
SELECT
    ROUND(MAX(population / surfacearea)::NUMERIC, 2) AS "Max pop_density",
    ROUND(AVG(population / surfacearea)::NUMERIC, 2) AS "Average pop_density",
    ROUND(MIN(population / surfacearea)::NUMERIC, 2) AS "Min pop_density"
FROM
    country
WHERE
    continent = 'Europe'
;
"""

pd.read_sql_query(query, con=conn)

,Max pop_density,Average pop_density,Min pop_density
0,22666.67,769.42,0.05


### 1.3

rubric={reasoning:1}

We'd like to write a query to return the name of the country with the greatest surface area in the `world_dsci513` database. Would the following query work as expected? Explain your answer in a 2-3 sentences.

```sql
SELECT
    name, MAX(surfacearea)
FROM
    country
```

No, it will not, because it's not possible to have both aggregated and non-aggregated columns in a single query unless the non-aggregated columns also appear in the `GROUP BY` clause.

### 1.4 (OPTIONAL)

rubric={accuracy:2}

Can you write a query to answer the question posed in [Exercise 1.3](#1.3)? Your result should contain one column and one row containing the value described above.

> **Hint:** Surprisingly, you don't need to use aggregation!

In [6]:
query = """
SELECT
    name
FROM
    country
ORDER BY
    surfacearea DESC
LIMIT 1
;
"""

pd.read_sql_query(query, con=conn)

,name
0,Russian Federation


### 1.5

rubric={accuracy:2}

Write a query that returns the total population of each region of the world according to the `country` table.

- Sort your results in descending order by each region's total population (Hint: Be careful not to sort alphabetically!).
- In order to increase the readability of the results, use the `to_char()` function to separate groups of thousands with commas
(see the documentation [here](https://www.postgresql.org/docs/current/functions-formatting.html#FUNCTIONS-FORMATTING-NUMERIC-TABLE)).
For example, to accommodate numbers going up to a billion, you can use `to_char(column, '9,999,999,999')`.

In [7]:
query = """
SELECT
    region,
    to_char(SUM(population), '9,999,999,999') AS sum_pop
FROM
    country
GROUP BY
    region
ORDER BY
    SUM(population) DESC
;
"""

pd.read_sql_query(query, con=conn)

,region,sum_pop
0,Eastern Asia,"1,507,328,000"
1,Southern and Central Asia,"1,490,776,000"
2,Southeast Asia,"518,541,000"
3,South America,"345,780,000"
4,North America,"309,632,000"
5,Eastern Europe,"307,026,000"
6,Eastern Africa,"246,999,000"
7,Western Africa,"221,672,000"
8,Middle East,"188,380,700"
9,Western Europe,"183,247,600"


### 1.6

rubric={accuracy:2}

What is the number of countries in each region that have a republic form of government? Sort your results by the number of countries in descending order.

In [8]:
query = """
SELECT
    region,
    COUNT(governmentform)
FROM
    country
WHERE
    governmentform ~ '(?i)republic'
GROUP BY
    region
ORDER BY
    COUNT(governmentform) DESC
;
"""

pd.read_sql_query(query, con=conn)

,region,count
0,Eastern Africa,17
1,Western Africa,16
2,South America,12
3,Southern and Central Asia,11
4,Southern Europe,11
5,Eastern Europe,10
6,Middle East,10
7,Central Africa,9
8,Central America,7
9,Southeast Asia,6


### 1.7 (OPTIONAL)

rubric={accuracy:2}

Using the `countrylanguage` table, write a query to find the `countrycode` and number of spoken languages in countries where 

- Each listed language is spoken by at least 10% of the population,
- There are at least 2 spoken languages in those countries.

Sort the resulting rows by the number of listed languages in each country in descending order.

In [9]:
query = """
SELECT
    countrycode,
    COUNT(*) AS num_lang
FROM
    countrylanguage
GROUP BY
    countrycode
HAVING
    MIN(percentage) >= 10
    AND
    COUNT(*) > 1
ORDER BY
    COUNT(*) DESC
;
"""

pd.read_sql_query(query, con=conn)

,countrycode,num_lang
0,GAB,4
1,VUT,3
2,DJI,3
3,LKA,3
4,MYT,3
5,NCL,3
6,BTN,3
7,MAR,2
8,PRI,2
9,LCA,2


### 1.8

rubric={accuracy:2}

Write a query to find the `countrycode` of countries that have at least 3 official languages. To verify your results, also return the number of official languages and name the corresponding column `num_official_lang`. Sort the returned rows according to this column in descending order.

In [10]:
query = """
SELECT
    countrycode,
    COUNT(*) AS num_official_lang
FROM
    countrylanguage
WHERE
    isofficial = TRUE
GROUP BY
    countrycode
HAVING
    COUNT(*) >= 3
ORDER BY
    num_official_lang DESC
;
"""

pd.read_sql_query(query, con=conn)

,countrycode,num_official_lang
0,CHE,4
1,ZAF,4
2,SGP,3
3,BOL,3
4,LUX,3
5,BEL,3
6,PER,3
7,VUT,3


## Exercise 2: Why don't you `JOIN` us?
---

### 2.1

rubric={accuracy:2}

It's hard to figure out which countries we're talking about in Exercise [1.8](#1.8) just by looking at their codes. Copy the query you wrote in Exercise [1.8](#1.8) here, and modify it such that it returns the name of each country instead of country code.

>**Hint:** You need to join the `country` and `countrylanguage` tables.

In [11]:
query = """
SELECT
    c.name,
    COUNT(*) AS num_official_lang
FROM
    countrylanguage AS cl
JOIN
    country AS c
ON
    c.code = cl.countrycode
WHERE
    cl.isofficial = TRUE
GROUP BY
    c.name
HAVING
    COUNT(*) >= 3
ORDER BY
    COUNT(*) DESC
;
"""

pd.read_sql_query(query, con=conn)

,name,num_official_lang
0,Switzerland,4
1,South Africa,4
2,Vanuatu,3
3,Belgium,3
4,Luxembourg,3
5,Peru,3
6,Bolivia,3
7,Singapore,3


### 2.2

rubric={accuracy:2}

Write a query that finds the ratio of the population of each country's capital city to its entire population shown as a percentage value, for countries that have a population of at least 1,000,000.

- Your query should list the country name, capital city, and the population ratio percentage value
- Name the population ratio column `pop_ratio`, and round the values to 1 decimal digit
- Sort your results in descending order by the population percentage
- Limit the number of returned countries to 20 in your SQL query

> **Hint:** Watch out for integer division; use type conversion if needed.

In [12]:
query = """
SELECT
    c.name "Country",
    ci.name "Capital",
    ROUND(ci.population::NUMERIC / c.population * 100, 1) pop_ratio
FROM
    country c
JOIN
    city ci
ON
    c.capital = ci.id
WHERE
    c.population >= 1000000
ORDER BY
    ROUND(ci.population::NUMERIC / c.population * 100, 1) DESC
LIMIT 20
;
"""

pd.read_sql_query(query, con=conn)

,Country,Capital,pop_ratio
0,Singapore,Singapore,112.6
1,Uruguay,Montevideo,37.0
2,Armenia,Yerevan,35.5
3,Gabon,Libreville,34.2
4,Lebanon,Beirut,33.5
5,Congo,Brazzaville,32.3
6,Latvia,Riga,31.5
7,Chile,Santiago de Chile,30.9
8,Libyan Arab Jamahiriya,Tripoli,30.0
9,Mongolia,Ulan Bator,29.1


### 2.3

rubric={accuracy:2}

Write a query that lists country names along with the average population and number of their listed cities.

- Pick appropriate names for the columns in your results. They can be anything you like.
- Use `to_char()` (which you've learned in a previous exercise in this lab) to format the average populations such that groups of thousands are separated by commas, and decimal digits are eliminated. For example, 1656782.25 should be shown as 1,656,782.
- Sort the results by the number of cities in each country in descending order
- limit the number of returned rows to 20.

In [13]:
query = """
SELECT
    c.name "Country",
    to_char(AVG(ci.population), '999,999,999') "Average population",
    COUNT(ci.name) "Number of cities"
FROM
    country c
JOIN
    city ci
ON
    c.code = ci.countrycode
GROUP BY
    c.name
ORDER BY
    "Number of cities" DESC
LIMIT 20
;
"""

pd.read_sql_query(query, con=conn)

,Country,Average population,Number of cities
0,China,"484,721",363
1,India,"361,579",341
2,United States,"286,955",274
3,Brazil,"343,507",250
4,Japan,"314,375",248
5,Russian Federation,"365,877",189
6,Mexico,"345,390",173
7,Philippines,"227,462",136
8,Germany,"282,209",93
9,Indonesia,"441,008",85


### 2.4 (OPTIONAL)

rubric={accuracy:2}

Write a query that returns country names, and their region and population along with

- the number of official languages in each country
- the number of cities of each country having a population of over 1 million

for countries that have at least 1 official language AND 1 city with a population of over 1 million.

Make sure that

- Give appropriate names to the additional columns
- Sort your results in descending order by the number of official languages in each country

> **Hint:** Since you need to do 3 joins, you'll get a lot of duplicates. Make sure to count only unique values!

In [14]:
query = """
SELECT
    co.name,
    co.region,
    co.population,
    COUNT(DISTINCT cl.language) AS "# of official languages", 
    COUNT(DISTINCT ci.name) AS "# of cities with pop > 1 million"
FROM
    country co
JOIN
    city ci
ON
    co.code = ci.countrycode
JOIN
    countrylanguage cl
ON
    co.code = cl.countrycode
WHERE
    ci.population > 1000000
    AND
    cl.isofficial = TRUE
GROUP BY
    co.code, co.region
ORDER BY
    "# of official languages" DESC
;
"""

pd.read_sql_query(query, con=conn)

,name,region,population,# of official languages,# of cities with pop > 1 million
0,South Africa,Southern Africa,40377000,4,1
1,Singapore,Southeast Asia,3567000,3,1
2,Peru,South America,25662000,3,1
3,Afghanistan,Southern and Central Asia,22720000,2,1
4,Romania,Eastern Europe,22455500,2,1
...,...,...,...,...,...
60,Armenia,Middle East,3520000,1,1
61,Australia,Australia and New Zealand,18886000,1,4
62,Austria,Western Europe,8091800,1,1
63,Azerbaijan,Middle East,7734000,1,1


### 2.5

rubric={accuracy:3}

Now that we've learned about grouping, aggregation and joins, let's revisit the last problem of Lab 1 and try to arrive at the same result using pure SQL. I hope after writing this query entirely in SQL, you'll appreciate the convenience of extracting this kind of information, compared to how you've done it in Pandas!

So without further ado, let's write a query that answers this question: what are the 10 most (natively) spoken languages in the world?

- Each row should show the language and the respective speaker population
- Sort your results by the second column in descending order
- Format the population numbers such that groups of thousands are separated with commas (you've already learned how to do this in previous exercises of this lab)
- Use appropriate column aliases that you like

Verify that you get exactly the same result as that you've arrived at in Lab 1 using Pandas!

In [15]:
query = """
SELECT
    cl.language,
    to_char(SUM(cl.percentage / 100 * c.population), '9,999,999,999') speaker_population
FROM
    country c
JOIN
    countrylanguage cl
ON
    c.code = cl.countrycode
GROUP BY
    cl.language
ORDER BY
    SUM(cl.percentage * c.population) DESC
LIMIT 10
;
"""

pd.read_sql_query(query, con=conn)

,language,speaker_population
0,Chinese,"1,191,843,539"
1,Hindi,"405,633,085"
2,Spanish,"355,029,462"
3,English,"347,077,861"
4,Arabic,"233,839,240"
5,Bengali,"209,304,713"
6,Portuguese,"177,595,269"
7,Russian,"160,807,560"
8,Japanese,"126,814,106"
9,Punjabi,"104,025,372"


## Exercise 3: More joins with the IMDB database
---

In this exercise, you'll explore the `imdb_dsci513` database more in depth and extract richer information by pulling data from various tables and joining them together.

In [16]:
conn = psycopg2.connect(database='imdb_dsci513', **login)

### 3.1

rubric={accuracy:2}

Write a query that returns the names of all actors/actresses of the movie **"Catch Me If You Can"**.

**Hint:** The data you need for this exercise is spread across the `movies`, `acting_roles`, and `names` tables.

In [17]:
query = """
SELECT
    m.title, n.name
FROM
    movies m
JOIN
    acting_roles a
ON
    m.id = a.movie_id
JOIN
    names n
ON
    a.name_id = n.id
WHERE
    m.title = 'Catch Me If You Can'
;
"""

pd.read_sql_query(query, con=conn)

,title,name
0,Catch Me If You Can,Grant Heslov
1,Catch Me If You Can,Matt Lattanzi
2,Catch Me If You Can,Loryn Locklin
3,Catch Me If You Can,Billy Morrissette
4,Catch Me If You Can,Leonardo DiCaprio
5,Catch Me If You Can,Tom Hanks
6,Catch Me If You Can,Martin Sheen
7,Catch Me If You Can,Christopher Walken


### 3.2

rubric={accuracy:2}

Write a query that lists each movie genre along with the average runtime of movies belonging to each genre. Sort your results in descending order by the latter column.

In [18]:
query = """
SELECT
    g.genre,
    AVG(m.runtime)
FROM
    movies m
JOIN
    movie_genres g
ON
    m.id = g.movie_id
GROUP BY
    g.genre
ORDER BY
    AVG(m.runtime) DESC
;
"""

pd.read_sql_query(query, con=conn)

,genre,avg
0,musical,123.522901
1,history,120.521104
2,war,115.552511
3,action,115.014854
4,biography,114.280488
5,drama,110.866650
6,romance,110.317226
7,thriller,109.705238
8,crime,108.376855
9,mystery,108.229765


### 3.3

rubric={accuracy:3}

Write a query to find the number of "drama" and "biography" movies in which "Marlon Brando", "Gary Oldman", and "Robin Williams" played a role. Your query should list the actor's name, genre, and number of movies played by that actor in that genre.

In [19]:
query = """
SELECT
    n.name, g.genre, COUNT(DISTINCT a.movie_id)
FROM
    names n
JOIN
    acting_roles a
ON
    a.name_id = n.id
JOIN
    movie_genres g
ON
    g.movie_id = a.movie_id
WHERE
    n.name IN ('Marlon Brando', 'Gary Oldman', 'Robin Williams')
    AND
    g.genre IN ('drama', 'biography')
GROUP BY
    n.name, g.genre
;
"""

pd.read_sql_query(query, con=conn)

,name,genre,count
0,Gary Oldman,biography,5
1,Gary Oldman,drama,23
2,Marlon Brando,biography,4
3,Marlon Brando,drama,27
4,Robin Williams,biography,5
5,Robin Williams,drama,23


## Exercise 4: Unter is the new Uber
---

In this exercise, we're going to create a database called `unter` and its tables from scratch, and then populate its tables with some fake data in the later exercises. The database `unter` is supposed to store data of employees, drivers, cars, etc. of a company which provides taxi services. Let's call our company _Unter_, because we want to be a rival to Uber!

(Uber in German means "over" or "above", so I've chosen "Unter" meaning "under" or "below" to oppose and compete with them even in name. But it's kinda obvious where the company's fate is headed with this name choice 😄)


**Please do this exercise on your local installation of Postgres, NOT on the remote server!**

Because you might want to drop your database several times as you try out new things that you've learned and you probably want to start fresh each time, I thought it may not be convenient to do it every time using the pgAdmin GUI. So I've given you the following cell to be able to **drop your database `unter` forcefully** (regardless if there are connections to it or not), and re-create it immediately.

In [20]:
conn = psycopg2.connect(database='postgres', **login)

autocommit = psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
conn.set_isolation_level(autocommit)
conn.cursor().execute("DROP DATABASE IF EXISTS unter WITH (FORCE);")
conn.cursor().execute("CREATE DATABASE unter;")
conn.cursor().close()

At this point, you can use pgAdmin to see that your new database `unter` is created. Don't forget to right-click your "Databases" group in the browser pane and click "Refresh". Alternatively, you can use the following cell to see if the database `unter` appears in the list of databases on your Postgres server:

In [21]:
conn = psycopg2.connect(database='postgres', **login)

with conn, conn.cursor() as cur:
    cur.execute("SELECT datname FROM pg_database;")
    print([i[0] for i in cur.fetchall()])

['postgres', 'template1', 'template0', 'world_dsci513', 'imdb_dsci513', 'mds', 'unter']


### 4.1

rubric={accuracy:3}

Now that you've created the database, it's time to create the tables we need. I've included `employee.csv` in the `data` folder of this assignment, which contains the data of employees of our company. Given this CSV file, your task is:

- Create a new table and call it `employee`
- Create an auto-incrementing column called `id` (Hint: Remember the `SERIAL` data type?). This is the primary-key column.
- Create other columns in your table that correspond to the columns you see in `employee.csv`. Use the same column names as in the CSV file.
- You also need to specify data types for your columns. To do this, inspect `employee.csv` and choose the most appropriate data types according to the values you see.
- Except for `exit_date`, none of the columns can be `NULL`.
- Values in column `sin` should be unique.
- The number of digits stored in column `sin` should be exactly 9 (Hint: Use the `CHECK` keyword together with `LENGTH()`).

> **Note:** Use your best judgment to choose the number of characters to allow in, for example, `VARCHAR(n)`. Just make sure that it's not unreasonably short or long. In any other case, it's arbitrary and up to you what length you use.

Make sure to enforce the constraints in your table. I've provided starter Python code for using the `psycopg2` package to do this:

In [22]:
conn = psycopg2.connect(database='unter', **login)
conn.autocommit = True

with conn, conn.cursor() as cur:
    cur.execute("""
    CREATE TABLE employee(
        id SERIAL,
        first_name VARCHAR(32) NOT NULL,
        last_name VARCHAR(64) NOT NULL,
        birth_date DATE NOT NULL,
        phone VARCHAR(32) NOT NULL,
        home_address VARCHAR(128) NOT NULL,
        home_postal_code VARCHAR(10) NOT NULL,
        home_city VARCHAR(32) NOT NULL,
        hire_date DATE NOT NULL,
        exit_date DATE,
        sin INT NOT NULL UNIQUE,

        CONSTRAINT employee_pk
            PRIMARY KEY (id)
    );
    """)

After creating your table, run `\d employee` in `psql` to see if your **columns** and **constraints** are properly created.

### 4.2

rubric={accuracy:2}

Being able to create things feels powerful, so let's create more!

Now let's create a table `driver` for the drivers who work at Unter. I've provided a data file called `driver.csv` in the `data` folder of this assignment. If you inspect this file, you'll see that the data about our drivers is pretty much similar to our employees data. The only difference is that in `driver.csv` we also store driver's license information, in addition to other columns in the `employee` table.

- You can use the same commands that you've written to create the `employee` table, but make sure to add more columns to accommodate the data in `driver.csv`.
- None of the new columns can be null.
- Add a constraint to the `driver_license` column so that its values are unique.

In [23]:
conn = psycopg2.connect(database='unter', **login)
conn.autocommit = True

with conn, conn.cursor() as cur:
    cur.execute("""
    CREATE TABLE driver(
        id SERIAL,
        first_name VARCHAR(32) NOT NULL,
        last_name VARCHAR(64) NOT NULL,
        birth_date DATE NOT NULL,
        phone VARCHAR(32) NOT NULL,
        home_address VARCHAR(128) NOT NULL,
        home_postal_code VARCHAR(10) NOT NULL,
        home_city VARCHAR(32) NOT NULL,
        hire_date DATE NOT NULL,
        exit_date DATE,
        sin INT NOT NULL UNIQUE,
        
        driver_licence CHAR(11) NOT NULL UNIQUE,
        issue_date DATE NOT NULL,
        expiry_date DATE NOT NULL,

        CONSTRAINT driver_pk
            PRIMARY KEY (id)
    );
    """)

### 4.3 (OPTIONAL)

rubric={accuracy:2}

After creating the `driver` table, we decide to add the following constraints in order to avoid having incorrect values inserted into the table..

- The date stored in `issue_date` can't be later than the date on which a row is inserted
- The date stored in `expiry_date` column can't be earlier than the `issue_date`

Use the `ALTER` command and the following cell to add the above constraints to the `driver` table.

In [24]:
conn = psycopg2.connect(database='unter', **login)
conn.autocommit = True

with conn, conn.cursor() as cur:
    cur.execute("""
        ALTER TABLE driver ADD
            CONSTRAINT issue_check CHECK (issue_date <= CURRENT_DATE);
            
        ALTER TABLE driver ADD
            CONSTRAINT expiry_check CHECK (expiry_date >= issue_date);
    """)

### 4.4

rubric={accuracy:2}

All right, almost there. We need to create two more tables, `car_model` and `cab`. `car_model` stores information about each type of car, whereas `cab` contains data about the particular cabs that the drivers of our company own. 

This time I've given you part of the `CREATE TABLE` statements for both tables, and you're in charge of adding constraints.

**`car_model` table:**

- Add the primary key constraint

**`cab` table:**

- Add the primary key constraint
- Each cab is a particular type of car, the information of which can be found in the `car_model` table. In order to ensure that each cab in the `cab` table can only be of the car types in the `car_model` table, the `car_model_id` column in the `cab` table should reference the `id` column of the `car_model` table. Add a constraint that enforces this.
- The owner of each can in the `cab` table should be one of the drivers of our company. Add a constraint to the `cab` table such that `owner_id` references the `id` column of the `driver` table.

In [25]:
conn = psycopg2.connect(database='unter', **login)
conn.autocommit = True

with conn, conn.cursor() as cur:
    cur.execute("""
    CREATE TABLE car_model(
        id SERIAL,
        model_name VARCHAR(64) NOT NULL,
        miles_per_gallon REAL,
        year DATE,
        origin VARCHAR(32),

        CONSTRAINT cars_model_pk
            PRIMARY KEY (id)
    );

    CREATE TABLE cab(
        id SERIAL,
        licence_plate VARCHAR(32) UNIQUE NOT NULL,
        car_model_id INT,
        owner_id INT,
        active BOOLEAN NOT NULL,

        CONSTRAINT cabs_pk
            PRIMARY KEY (id),

        CONSTRAINT cabs_fk_cars_model
            FOREIGN KEY (car_model_id)
                REFERENCES car_model(id),

        CONSTRAINT cabs_fk_owner
            FOREIGN KEY (owner_id)
                REFERENCES driver(id)
    );
    """)

## Exercise 5: Sanitize your SQL too, not just your hands
---

Some evil people have gotten really unhappy with the progress we've made with our amazing taxi company "Unter", so they hire a hacker to attack our company's database. Suppose that we have a website where our employees can log in and access their information. In order to give access, we have a table called `user` that stores employees/clients username and passwords. So when a user tries to log in, we do the authentication and retrieve that user's information using the following query

```python
with conn, conn.cursor() as cur:
    cur.execute(f"SELECT * FROM users WHERE username = '{emp_user}' AND password = '{emp_passwd}';")
```

Embedding strings directly in a SQL query using

- Python's f-strings (e.g. `f"I have {variable} apples"`) or
- Python's string concatenation (e.g. `"I have "+ variable + " apples"`)

is **EXTREMELY DANGEROUS, and must always be avoided!**. It makes your application/service vulnerable to a particular security breach called [**SQL injection attacks**](https://en.wikipedia.org/wiki/SQL_injection).

See what `psycopg2` documentation says about this:
<img src="img/warning.png" width="800">

But why? I'll show you!

With the above embedding of variables directly into the string that we send to the database server, a hacker who is well-seasoned in SQL can carefully craft a username or password string like this:

```python
emp_user = "' OR 1 = 1; SELECT * FROM user; --"
```

Now, what your database receives as a query is this:

```sql
SELECT * FROM users WHERE username = '' OR 1 = 1; SELECT * FROM user; -- ' AND password = '{emp_passwd}';")
```

Since `OR 1 = 1` is always true, the `WHERE` condition is short-circuited, and `SELECT * FROM user;` will get executed, while the rest of the query is neglected because of the `--`! Now the hacker has access to the username and passwords of all users stored in the `user` tables.

**How to avoid SQL injection attack?**

There are a few ways we can make sure that our application/service is not vulnerable to SQL injection attacks, one of which is to use **parameterized statements**. In `psycopg2`, you can safely embed variables inside your SQL statements using `%s` as parameter placeholders, and the second argument of the `.execute()` method to pass the parameters securely to the query ([see here for details](https://www.psycopg.org/docs/usage.html#passing-parameters-to-sql-queries)). For example, we can safely write our original query as:

```python
with conn, conn.cursor() as cur:
    cur.execute("SELECT * FROM users WHERE username = %s AND password = %s;",
               (emp_user, emp_passwd))
```

The above SQL statement is now a **sanitized** query, and can be safely executed. In this manner, the query and data are sent to the server separately. By the time data arrives, the database server already knows what the query exactly is. This means that the query cannot be changed no matter what the passed data is.

Furthermore, the parameters are checked before being used in the SQL statement by using `%` placeholders. For example, a single quote `'` in a parameter never gets a chance to close another single quote in the query, because it will be replaced with `''` and will therefore be escaped!

### 5.1

rubric={accuracy:2}

Suppose that you're in charge of assessing the security of the `unter` database, and you want to test to see if you can actually break the database on purpose by injecting a SQL statement. I've defined a sample row for you in the cell below, and also a VERY DANGEROUS and naively formed SQL statement ☠️ (you should never write a SQL query in Python like this!).

Your task is to try and create a value for one of the columns such that when the SQL statement is formed, it deletes the `driver` table!

**Hint:** It's easier to use the last column for your SQL injection.

**Hint:** Remember that other tables depend on the `driver` table, so unless you address this properly in your SQL statement, you won't be able to delete the table.

**Hint:** Don't forget to use double quotes for your value so that the single quotes in the Python code do not interfere with your SQL strings.

In [26]:
row = ['Bartel', 'Dall', '1997-08-15',
       '102-380-7032', '86 Vermont Park', 'G5T 7X1',
       'North Vancouver', '2018-06-05', '2020-02-01',
       '200887794', '100-86-4796', '2019-05-05', "2024-5-5'); DROP TABLE driver CASCADE;--'"]

(first_name, last_name, birth_date, phone, home_address,
 home_postal, home_city, hire_date, exit_date, sin,
 driver_licence, issue_date, expiry_date) = row


with conn, conn.cursor() as cur:
    cur.execute(f"""
        INSERT INTO driver VALUES
            (DEFAULT,
            '{first_name}', '{last_name}', '{birth_date}', '{phone}', '{home_address}',
            '{home_postal}', '{home_city}', '{hire_date}', '{exit_date}', '{sin}',
            '{driver_licence}', '{issue_date}', '{expiry_date}');
                """)

Well, this is a security disaster 😵‍💫

To make sure that the `driver` table is actually deleted as a result of your SQL injection, run the following cell to list tables in your `unter` database:

In [27]:
with conn, conn.cursor() as cur:
    cur.execute("""SELECT table_name FROM information_schema.tables WHERE table_schema='public'""")
    print([i[0] for i in cur.fetchall()])

['employee', 'car_model', 'cab']


Well, I hope you agree with me now how dangerous it is to expose your database like this. Always sanitize your queries!

### 5.2

rubric={accuracy:5}

Now that you've learned how to avoid the risk of embedding strings directly in your SQL statement, let's practice populating your `employee` table from within Python properly and safely in this exercise using the data in `employee.csv` file.

In order to form safe SQL statements in Python, you need to use `psycopg2`s placeholders `%` for embedding parameters in your SQL statements, instead of using f-strings or string concatenation (see [here](https://www.psycopg.org/docs/usage.html#passing-parameters-to-sql-queries)) for details.

I've provided some starter code for you that reads the `employee.csv` line by line and stores the column values in intermediate variables. You can use those variables to form your `INSERT` statements inside the loop.

In [28]:
with conn, conn.cursor() as cur:
    
    with open('data/employee.csv', 'r') as f:
        reader = csv.reader(f, delimiter=',')
        next(reader)  # skip the header row

        for row in reader:
            
            (first_name, last_name, birth_date, phone,
             home_address, home_postal, home_city,
             hire_date, exit_date, sin) = row
            
            if exit_date == '': exit_date = None
            
            cur.execute(
                "INSERT INTO employee VALUES (DEFAULT, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);",
                (first_name, last_name, birth_date, phone,
                 home_address, home_postal, home_city,
                 hire_date, exit_date, sin)
            )
            

Let's see if the table `employee` is populated:

In [29]:
pd.read_sql_query("SELECT * FROM employee LIMIT 5;", con=conn)

,id,first_name,last_name,birth_date,phone,home_address,home_postal_code,home_city,hire_date,exit_date,sin
0,1,Sylvia,Bolstridge,1986-09-16,279-727-2485,3717 Redwing Parkway,T8L 2J6,Surrey,2018-04-23,2019-06-29,224705842
1,2,Marmaduke,Pethybridge,1989-08-27,490-248-0001,0314 Dayton Way,E9C 7X7,Richmond,2019-10-14,None,285858804
2,3,Goldy,Robeiro,1985-11-21,117-477-3289,3385 Tennessee Circle,B2G 7L3,Maple Ridge,2020-05-17,None,110904388
3,4,Emelina,Mulholland,1987-10-15,779-269-2569,1 Atwood Circle,E5R 3H6,Surrey,2020-06-26,2019-03-11,605946081
4,5,Natka,Sleford,1964-12-15,381-367-9439,30050 Clemons Pass,T8R 6C4,Langley,2017-07-18,None,970542883


### 5.3 (OPTIONAL)

rubric={accuracy:1}

So far, you've learned how to create sanitized and safe SQL `INSERT` statements to read data from external files and load them into your database using `psycopg2` in Python. Let's now practice how to import data into the tables of your database using the `\copy` meta-command in Postgres that you can run from `psql`.

Your taks is to use the the `\copy` meta-command to import the data stored in the `car_model.csv` file in the `data` folder of this assignment into your `car_model` table. Here is a [link](https://www.postgresql.org/docs/10/app-psql.html#APP-PSQL-META-COMMANDS-COPY) to Postgres documentation for the `\copy` command. Feel free to look up the internet to see how you can form the appropriate `\copy` command for this exercise.

**Hint:** In `psql`, first navigate to the directory where your Lab 2 files are located before trying your command.

**Hint:** If your `\copy` command is successfully run, you'll see `COPY 406` in the output of `psql`.

```sql
\copy car_model(model_name, miles_per_gallon, year, origin) FROM 'data/car_model.csv' WITH DELIMITER ',' CSV HEADER;
```

Let's check if the data is properly imported:

In [369]:
pd.read_sql_query("SELECT * FROM car_model LIMIT 10;", con=conn)

,id,model_name,miles_per_gallon,year,origin
0,1,chevrolet chevelle malibu,18.0,1970-01-01,USA
1,2,buick skylark 320,15.0,1970-01-01,USA
2,3,plymouth satellite,18.0,1970-01-01,USA
3,4,amc rebel sst,16.0,1970-01-01,USA
4,5,ford torino,17.0,1970-01-01,USA
5,6,ford galaxie 500,15.0,1970-01-01,USA
6,7,chevrolet impala,14.0,1970-01-01,USA
7,8,plymouth fury iii,14.0,1970-01-01,USA
8,9,pontiac catalina,14.0,1970-01-01,USA
9,10,amc ambassador dpl,15.0,1970-01-01,USA


### 5.4 But it's no joke

As a reward for completing Exercise 5, here is a cartoon ([source](https://xkcd.com/327/)) that only SQL people understand:

<img src="img/cartoon.png" width="800">